Importación y Carga de Datos

In [7]:
# Importaciones correctas
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras_tuner import Hyperband
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from xgboost import XGBClassifier
from keras_tuner import Hyperband

# Carga de datos
df = pd.read_csv('Telco Churn dataset 2.csv')

Preprocesamiento

In [8]:
# 1. Preprocesamiento
# a. Crear la columna 'Churn' y asignar 1 si 'Churn' es 'Yes', de lo contrario 0
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# b. Eliminar columnas con más del 50% de datos faltantes
threshold = int(0.5 * len(df))
df = df.dropna(thresh=threshold, axis=1)

# c. Reemplazar valores atípicos por la media
for col in df.select_dtypes(include=['float64', 'int64']):
    if col != 'Churn':  # Asegurarse de no modificar la columna 'Churn'
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        df[col] = df[col].apply(lambda x: df[col].mean() if (x < (Q1 - 1.5 * IQR)) or (x > (Q3 + 1.5 * IQR)) else x)

# d. Convertir variables categóricas a numéricas y llenar valores faltantes
for column in df.columns:
    if df[column].dtype == 'object' and column != 'Churn':
        df[column].fillna(df[column].mode()[0], inplace=True)
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
    elif df[column].dtype in ['int64', 'float64']:
        df[column].fillna(df[column].median(), inplace=True)

# e. Normalización
cols_to_scale = df.columns.tolist()
cols_to_scale.remove('Churn')
scaler = StandardScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

# f. Asegurarse de que 'Churn' sea int
df['Churn'] = df['Churn'].astype(int)


Selección de características - Gradient Boosting

In [9]:
# 2. Selección de características
xgb_for_feature_selection = XGBClassifier(
    objective='binary:logistic', 
    random_state=42, 
    use_label_encoder=False, 
    eval_metric='logloss'
)
xgb_for_feature_selection.fit(df.drop('Churn', axis=1), df['Churn'])
threshold = 0.01  
selected_features = df.drop('Churn', axis=1).columns[(xgb_for_feature_selection.feature_importances_ > threshold)].tolist()
print("Características seleccionadas:", selected_features)

Características seleccionadas: ['customerID', 'SeniorCitizen', 'tenure', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'InternationalPlan', 'VoiceMailPlan', 'NumbervMailMessages', 'TotalDayMinutes', 'TotalEveMinutes', 'TotalNightMinutes', 'TotalIntlMinutes', 'TotalIntlCalls', 'CustomerServiceCalls']


Balanceo con SMOTE

In [10]:
# 3. Balanceo con SMOTE
X = df[selected_features]
y = df['Churn']
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

División de ConjuntoBalanceo

In [11]:
# 4. División de Conjunto
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)
X_train_cnn = np.reshape(X_train.values, (X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = np.reshape(X_test.values, (X_test.shape[0], X_test.shape[1], 1))

Entrenamiento del modelo

In [12]:
# 5. Construcción del Modelo CNN para GridSearch
def build_model(hp):
    model = Sequential()
    model.add(Conv1D(
        filters=hp.Int('conv_filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Int('conv_kernel_size', min_value=2, max_value=5),
        activation='relu',
        input_shape=(X_train_cnn.shape[1], 1)
    ))
    model.add(MaxPooling1D(pool_size=hp.Int('max_pool_size', min_value=2, max_value=4)))
    model.add(Flatten())
    model.add(Dense(units=hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.legacy.Adam(hp.Choice('learning_rate', values=[0.001, 0.01, 0.1])),
        metrics=['accuracy']
    )
    return model

# Instantiate the tuner
tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    directory='hyperband',
    project_name='cnn'
)

# Perform hyperparameter tuning
tuner.search(X_train_cnn, y_train, epochs=10, validation_split=0.2, verbose=2)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
La búsqueda de hiperparámetros está completa. El número óptimo de unidades en la primera capa densamente conectada
es {best_hps.get('dense_units')} y la tasa de aprendizaje óptima para el optimizador
es {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 13s]
val_accuracy: 0.8453947305679321

Best val_accuracy So Far: 0.8892543911933899
Total elapsed time: 00h 04m 04s

La búsqueda de hiperparámetros está completa. El número óptimo de unidades en la primera capa densamente conectada
es 128 y la tasa de aprendizaje óptima para el optimizador
es 0.001.



Evaluación del modelo

In [13]:
# 6. Evaluación del modelo con los mejores hiperparámetros
# Obtén el mejor modelo
best_cnn_model = tuner.get_best_models(num_models=1)[0]

# Evalúa el modelo en el conjunto de test
loss_cnn, accuracy_cnn = best_cnn_model.evaluate(X_test_cnn, y_test)
print(f'Loss: {loss_cnn}, Accuracy: {accuracy_cnn}')

# Haz predicciones en el conjunto de test
y_pred_cnn = (best_cnn_model.predict(X_test_cnn) > 0.5).astype("int32")

# Calcula la matriz de confusión y el reporte de clasificación
conf_matrix_cnn = confusion_matrix(y_test, y_pred_cnn)
report_cnn = classification_report(y_test, y_pred_cnn)

# Imprime las métricas
print(conf_matrix_cnn)
print("Reporte de clasificación:\n", report_cnn)

36/36 [==============================] - 2s 34ms/step - loss: 0.2963 - accuracy: 0.8807
Loss: 0.2962915897369385, Accuracy: 0.8807017803192139
36/36 [==============================] - 1s 7ms/step
[[514  61]
 [ 75 490]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.87      0.89      0.88       575
           1       0.89      0.87      0.88       565

    accuracy                           0.88      1140
   macro avg       0.88      0.88      0.88      1140
weighted avg       0.88      0.88      0.88      1140

